# Importing Python Libraries

In [1]:
import requests
import json
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect('database.sqlite')
cur = conn.cursor()

In [13]:
cur.execute("""SELECT Match_ID, TeamName, HomeTeam, AwayTeam, Date, Season, Div, 
                FTHG AS HomeTeamGoals, FTAG AS AwayTeamGoals, FTR AS Winners
                FROM Matches
                JOIN Teams_in_Matches
                USING (Match_ID)
                JOIN Unique_Teams
                USING (Unique_Team_ID)
                WHERE Season = 2011
                ;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,Match_ID,TeamName,HomeTeam,AwayTeam,Date,Season,Div,HomeTeamGoals,AwayTeamGoals,Winners
0,1092,Bayern Munich,Nurnberg,Bayern Munich,2012-03-31,2011,D1,0,1,A
1,1092,Nurnberg,Nurnberg,Bayern Munich,2012-03-31,2011,D1,0,1,A
2,1093,Bayern Munich,Stuttgart,Bayern Munich,2011-12-11,2011,D1,1,2,A
3,1093,Stuttgart,Stuttgart,Bayern Munich,2011-12-11,2011,D1,1,2,A
4,1094,Bayern Munich,Wolfsburg,Bayern Munich,2011-08-13,2011,D1,0,1,A


In [14]:
all_games = len(df)/2
non_German_div_index = df[df['Div'] == "E0" ].index
df.drop(non_German_div_index , inplace=True)
german_games = len(df)/2
print("{} games removed!".format(int(all_games - german_games)))

380 games removed!


In [15]:
for i, row in df.iterrows():
    if row.TeamName == row.AwayTeam:
        if row.Winners == 'A':
            df.loc[i, 'Result'] = 'W'
        if row.Winners == 'D':
            df.loc[i, 'Result'] = 'D'
        if row.Winners == 'H':
            df.loc[i, 'Result'] = 'L'
    else:
        if row.Winners == 'H':
            df.loc[i, 'Result'] = 'W'
        if row.Winners == 'D':
            df.loc[i, 'Result'] = 'D'
        if row.Winners == 'A':
            df.loc[i, 'Result'] = 'L'

In [16]:
df.head()

,Match_ID,TeamName,HomeTeam,AwayTeam,Date,Season,Div,HomeTeamGoals,AwayTeamGoals,Winners,Result
0,1092,Bayern Munich,Nurnberg,Bayern Munich,2012-03-31,2011,D1,0,1,A,W
1,1092,Nurnberg,Nurnberg,Bayern Munich,2012-03-31,2011,D1,0,1,A,L
2,1093,Bayern Munich,Stuttgart,Bayern Munich,2011-12-11,2011,D1,1,2,A,W
3,1093,Stuttgart,Stuttgart,Bayern Munich,2011-12-11,2011,D1,1,2,A,L
4,1094,Bayern Munich,Wolfsburg,Bayern Munich,2011-08-13,2011,D1,0,1,A,W


In [17]:
for i, row in df.iterrows():
    if row.TeamName == row.AwayTeam:
        df.loc[i, 'GoalsScored'] = df.loc[i, 'AwayTeamGoals']
    elif row.TeamName == row.HomeTeam:
        df.loc[i, 'GoalsScored'] = df.loc[i, 'HomeTeamGoals']

In [18]:
df.head()

,Match_ID,TeamName,HomeTeam,AwayTeam,Date,Season,Div,HomeTeamGoals,AwayTeamGoals,Winners,Result,GoalsScored
0,1092,Bayern Munich,Nurnberg,Bayern Munich,2012-03-31,2011,D1,0,1,A,W,1.0
1,1092,Nurnberg,Nurnberg,Bayern Munich,2012-03-31,2011,D1,0,1,A,L,0.0
2,1093,Bayern Munich,Stuttgart,Bayern Munich,2011-12-11,2011,D1,1,2,A,W,2.0
3,1093,Stuttgart,Stuttgart,Bayern Munich,2011-12-11,2011,D1,1,2,A,L,1.0
4,1094,Bayern Munich,Wolfsburg,Bayern Munich,2011-08-13,2011,D1,0,1,A,W,1.0


In [19]:
total_goals_scored = df.pivot_table(index = 'TeamName', aggfunc='sum')
total_goals_scored = total_goals_scored.rename(columns={"GoalsScored":"Total Goals Scored (2011 Season)"})
total_goals_scored = total_goals_scored.drop(columns = ['AwayTeamGoals','HomeTeamGoals','Match_ID','Season'])
total_goals_scored.head()

,Total Goals Scored (2011 Season)
TeamName,
Aachen,30.0
Augsburg,36.0
Bayern Munich,77.0
Bochum,41.0
Braunschweig,37.0


In [23]:
wins_and_losses = df.pivot_table(df, index = 'TeamName', columns=['Result'], aggfunc='count')
wins_and_losses = wins_and_losses.drop(columns = ['AwayTeam', 'AwayTeamGoals', 'Date', 'Div','HomeTeamGoals','Match_ID','Season','GoalsScored','HomeTeam'])
wins_and_losses.head()

wins_and_losses.columns = [col[1] for col in wins_and_losses.columns]
wins_and_losses.reset_index()


,TeamName,D,L,W
0,Aachen,13,15,6
1,Augsburg,14,12,8
2,Bayern Munich,4,7,23
3,Bochum,7,17,10
4,Braunschweig,15,9,10
5,Cottbus,11,15,8
6,Dortmund,6,3,25
7,Dresden,9,13,12
8,Duisburg,9,15,10
9,Ein Frankfurt,8,6,20


In [ ]:
wins_and_

# Connecting to the DarkSky API

In [ ]:
def get_keys(path):
    with open(path) as f:
        return json.load(f)

In [ ]:
keys = get_keys("/Users/ravimalde/.secret/dark_sky_api.json")
api_key = keys['api_key']
print(api_key)

latitude = 51.5074
longitude = 0.1278

time = 1577836800

url = "https://api.darksky.net/forecast/{}/{},{}".format(api_key, latitude, longitude, time)

r = requests.get(url)

print(r.text)

In [ ]:
r.json().keys()

In [ ]:
r.json()['currently']